In [3]:
import json
import os

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure CUDA device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# short_datasets = "m4_yearly m4_quarterly m4_monthly m4_weekly m4_daily m4_hourly electricity/15T electricity/H electricity/D electricity/W solar/10T solar/H solar/D solar/W hospital covid_deaths us_births/D us_births/M us_births/W saugeenday/D saugeenday/M saugeenday/W temperature_rain_with_missing kdd_cup_2018_with_missing/H kdd_cup_2018_with_missing/D car_parts_with_missing restaurant hierarchical_sales/D hierarchical_sales/W LOOP_SEATTLE/5T LOOP_SEATTLE/H LOOP_SEATTLE/D SZ_TAXI/15T SZ_TAXI/H M_DENSE/H M_DENSE/D ett1/15T ett1/H ett1/D ett1/W ett2/15T ett2/H ett2/D ett2/W jena_weather/10T jena_weather/H jena_weather/D bitbrains_fast_storage/5T bitbrains_fast_storage/H bitbrains_rnd/5T bitbrains_rnd/H bizitobs_application bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"
# # short_datasets = "m4_weekly"
short_datasets = ""

# med_long_datasets = "electricity/15T electricity/H solar/10T solar/H kdd_cup_2018_with_missing/H LOOP_SEATTLE/5T LOOP_SEATTLE/H SZ_TAXI/15T M_DENSE/H ett1/15T ett1/H ett2/15T ett2/H jena_weather/10T jena_weather/H bitbrains_fast_storage/5T bitbrains_rnd/5T bizitobs_application bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"
# med_long_datasets = "bizitobs_l2c/H"
med_long_datasets = "electricity/H"

# Get union of short and med_long datasets
all_datasets = list(set(short_datasets.split() + med_long_datasets.split()))

dataset_properties_map = json.load(open("dataset_properties.json"))


from gluonts.ev.metrics import (
    MAE,
    MAPE,
    MASE,
    MSE,
    MSIS,
    ND,
    NRMSE,
    RMSE,
    SMAPE,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(
        quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    ),
]

import timesfm

# tfm = timesfm.TimesFm(
#     hparams=timesfm.TimesFmHparams(
#         backend="gpu",
#         per_core_batch_size=32,
#         num_layers=50,
#         horizon_len=128,
#         context_len=2048,
#         use_positional_embedding=False,
#         output_patch_len=128,
#     ),
#     checkpoint=timesfm.TimesFmCheckpoint(
#         huggingface_repo_id="google/timesfm-2.0-500m-jax"),
# )

# If you are using the pytorch version:
tfm = timesfm.TimesFm(
    hparams=timesfm.TimesFmHparams(
        backend="gpu",
        per_core_batch_size=32,
        num_layers=50,
        horizon_len=128,
        context_len=512,
        use_positional_embedding=False,
        output_patch_len=128,
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
)

from typing import List
import numpy as np
from tqdm.auto import tqdm
from gluonts.itertools import batcher
from gluonts.model import Forecast
from gluonts.model.forecast import QuantileForecast


class TimesFmPredictor:

  def __init__(
      self,
      tfm,
      prediction_length: int,
      ds_freq: str,
      *args,
      **kwargs,
  ):
    self.tfm = tfm
    self.prediction_length = prediction_length
    if self.prediction_length > self.tfm.horizon_len:
      self.tfm.horizon_len = (
          (self.prediction_length + self.tfm.output_patch_len - 1) //
          self.tfm.output_patch_len) * self.tfm.output_patch_len
      print('Jitting for new prediction length.')
    self.freq = timesfm.freq_map(ds_freq)

  def predict(self, test_data_input, batch_size: int = 1024) -> List[Forecast]:
    forecast_outputs = []
    for batch in tqdm(batcher(test_data_input, batch_size=batch_size)):
      context = []
      for entry in batch:
        arr = np.array(entry["target"])
        context.append(arr)
      freqs = [self.freq] * len(context)
      _, full_preds = self.tfm.forecast(context, freqs, normalize=True)
      full_preds = full_preds[:, 0:self.prediction_length, 1:]
      forecast_outputs.append(full_preds.transpose((0, 2, 1)))
    forecast_outputs = np.concatenate(forecast_outputs)

    # Convert forecast samples into gluonts Forecast objects
    forecasts = []
    for item, ts in zip(forecast_outputs, test_data_input):
      forecast_start_date = ts["start"] + len(ts["target"])
      forecasts.append(
          QuantileForecast(
              forecast_arrays=item,
              forecast_keys=list(map(str, self.tfm.quantiles)),
              start_date=forecast_start_date,
          ))

    return forecasts
  
import logging


class WarningFilter(logging.Filter):
    def __init__(self, text_to_filter):
        super().__init__()
        self.text_to_filter = text_to_filter

    def filter(self, record):
        return self.text_to_filter not in record.getMessage()


gts_logger = logging.getLogger("gluonts.model.forecast")
gts_logger.addFilter(
    WarningFilter("The mean prediction is not stored in the forecast data")
)

import csv
import os

from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality

from gift_eval.data import Dataset

all_ds_tuples = []

pretty_names = {
    "saugeenday": "saugeen",
    "temperature_rain_with_missing": "temperature_rain",
    "kdd_cup_2018_with_missing": "kdd_cup_2018",
    "car_parts_with_missing": "car_parts",
}

for ds_num, ds_name in enumerate(all_datasets):
  ds_key = ds_name.split("/")[0]
  print(f"Processing dataset: {ds_name} ({ds_num + 1} of {len(all_datasets)})")
  terms = ["short", "medium", "long"]
  for term in terms:
    if (term == "medium" or
        term == "long") and ds_name not in med_long_datasets.split():
      continue

    if "/" in ds_name:
      ds_key = ds_name.split("/")[0]
      ds_freq = ds_name.split("/")[1]
      ds_key = ds_key.lower()
      ds_key = pretty_names.get(ds_key, ds_key)
    else:
      ds_key = ds_name.lower()
      ds_key = pretty_names.get(ds_key, ds_key)
      ds_freq = dataset_properties_map[ds_key]["frequency"]
    ds_config = f"{ds_key}/{ds_freq}/{term}"
    # Initialize the dataset
    to_univariate = (False if Dataset(
        name=ds_name, term=term, to_univariate=False).target_dim == 1 else True)
    dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate)
    all_ds_tuples.append(
        (dataset.prediction_length, ds_config, ds_name, to_univariate))
    
all_ds_tuples = sorted(all_ds_tuples)
all_ds_tuples[0:10]


model_name = "timesfm_2_0_500m"
output_dir = f"../results_ref/{model_name}"
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "all_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
  writer = csv.writer(csvfile)

  # Write the header
  writer.writerow([
      "dataset",
      "model",
      "eval_metrics/MSE[mean]",
      "eval_metrics/MSE[0.5]",
      "eval_metrics/MAE[0.5]",
      "eval_metrics/MASE[0.5]",
      "eval_metrics/MAPE[0.5]",
      "eval_metrics/sMAPE[0.5]",
      "eval_metrics/MSIS",
      "eval_metrics/RMSE[mean]",
      "eval_metrics/NRMSE[mean]",
      "eval_metrics/ND[0.5]",
      "eval_metrics/mean_weighted_sum_quantile_loss",
      "domain",
      "num_variates",
  ])

for entry in all_ds_tuples:
  prediction_length = entry[0]
  ds_name = entry[2]
  to_univariate = entry[3]
  ds_config = entry[1]
  ds_key, ds_freq, term = ds_config.split("/")
  dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate)
  season_length = get_seasonality(dataset.freq)
  print(f"Processing entry: {entry}")
  print(f"Dataset size: {len(dataset.test_data)}")
  predictor = TimesFmPredictor(
      tfm=tfm,
      prediction_length=dataset.prediction_length,
      ds_freq=ds_freq,
  )
  # Measure the time taken for evaluation
  res = evaluate_model(
      predictor,
      test_data=dataset.test_data,
      metrics=metrics,
      batch_size=1024,
      axis=None,
      mask_invalid_label=True,
      allow_nan_forecast=False,
      seasonality=season_length,
  )

  # Append the results to the CSV file
  with open(csv_file_path, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        ds_config,
        model_name,
        res["MSE[mean]"][0],
        res["MSE[0.5]"][0],
        res["MAE[0.5]"][0],
        res["MASE[0.5]"][0],
        res["MAPE[0.5]"][0],
        res["sMAPE[0.5]"][0],
        res["MSIS"][0],
        res["RMSE[mean]"][0],
        res["NRMSE[mean]"][0],
        res["ND[0.5]"][0],
        res["mean_weighted_sum_quantile_loss"][0],
        dataset_properties_map[ds_key]["domain"],
        dataset_properties_map[ds_key]["num_variates"],
    ])

  print(f"Results for {ds_name} have been written to {csv_file_path}")

import pandas as pd

df = pd.read_csv(f"../results_ref/{model_name}/all_results.csv")
df

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 85948.85it/s]

Processing dataset: electricity/H (1 of 1)
Processing entry: (48, 'electricity/H/short', 'electricity/H', False)
Dataset size: 7400


8it [00:06,  1.31it/s]
7400it [00:33, 221.86it/s]


Results for electricity/H have been written to ../results_ref/timesfm_2_0_500m/all_results.csv
Processing entry: (480, 'electricity/H/medium', 'electricity/H', False)
Dataset size: 2960
Jitting for new prediction length.


3it [00:08,  2.88s/it]
2960it [00:13, 225.91it/s]


Results for electricity/H have been written to ../results_ref/timesfm_2_0_500m/all_results.csv
Processing entry: (720, 'electricity/H/long', 'electricity/H', False)
Dataset size: 1850
Jitting for new prediction length.


2it [00:08,  4.06s/it]
1850it [00:08, 221.04it/s]

Results for electricity/H have been written to ../results_ref/timesfm_2_0_500m/all_results.csv


,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,electricity/H/short,timesfm_2_0_500m,1.107471e+06,1.107471e+06,144.843216,0.808624,0.171820,0.114977,8.619365,1052.364624,0.496791,0.068376,0.056101,Energy,1
1,electricity/H/medium,timesfm_2_0_500m,6.909636e+06,6.909636e+06,248.510936,1.030874,0.237866,0.128217,14.448191,2628.618559,1.026036,0.097002,0.083306,Energy,1
2,electricity/H/long,timesfm_2_0_500m,1.154380e+07,1.154380e+07,297.802341,1.190219,0.315071,0.142651,21.081368,3397.616703,1.340019,0.117453,0.104020,Energy,1
